In [ ]:
# =============================================================================
# ШАБЛОН FINE-TUNING Qwen3-14B с Unsloth (максимальная скорость + минимум VRAM)
# Полностью готов к запуску на 1–2 GPU (даже на одной 4090 в 4-bit помещается!)
# Тестировалось в ноябре 2025
# =============================================================================

import torch
from datasets import load_dataset, DatasetDict
from trl import SFTTrainer
from transformers import TrainingArguments, set_seed
from unsloth import FastLanguageModel, is_bfloat16_supported
import gc
import json
from tqdm.auto import tqdm

# -------------------------- 1. НАСТРОЙКИ --------------------------
set_seed(42)

MODEL_NAME = "unsloth/Qwen3-14B"                  # именно unsloth-версия!
# Если хотите 32B — просто замените на unsloth/Qwen3-32B
OUTPUT_DIR = "./qwen3-14b-finetuned-unsloth"

MAX_SEQ_LENGTH = 32768          # Qwen3 поддерживает до 32k контекста!
BATCH_SIZE_PER_GPU = 2          # реальный батч на GPU
GRADIENT_ACCUMULATION_STEPS = 8
TOTAL_EPOCHS = 3
LEARNING_RATE = 2e-4
WARMUP_RATIO = 0.05

# Unsloth автоматически выберет оптимальные LoRA-параметры для Qwen3
# r=64 и alpha=16 — золотой стандарт для 14B–32B моделей в 2025
# target_modules unsloth сам найдёт (включая новые gate, up, down в Qwen3)

# -------------------------- 2. ЗАГРУЗКА МОДЕЛИ И ТОКЕНИЗАТОРА С UNSLOTH ----------
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,                  # None = автоопределение (bf16 на Ampere+, float16 на старых)
    load_in_4bit=True,           # ОБЯЗАТЕЛЬНО включаем 4-bit для 14B
    token="hf_...",              # если модель гейтированная — укажите свой HF токен
    # device_map="auto",        # unsloth сам управляет device_map
)

# Добавляем LoRA-адаптеры через unsloth (самый быстрый и экономный способ)
model = FastLanguageModel.get_peft_model(
    model,
    r=64,                        # ранг LoRA
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],  # все нужные для Qwen3
    lora_alpha=16,
    lora_dropout=0,              # unsloth оптимизировал — dropout=0 быстрее и не хуже
    bias="none",
    use_gradient_checkpointing="unsloth",  # супер-важная фича unsloth — экономит до 80% VRAM
    random_state=42,
    use_rslora=False,            # можно включить для чуть лучшего качества
    loftq_config=None,           # LoftQ не нужен для unsloth-версий
)

print(model.print_trainable_parameters())  # увидите ~1–1.5% обучаемых параметров

# -------------------------- 3. ДАТАСЕТ И ФОРМАТИРОВАНИЕ -----------------------
# Правильный chat_template для всех Qwen3
def formatting_func(example):
    messages = []
    
    # Если в датасете есть system-промпт
    if example.get("system"):
        messages.append({"role": "system", "content": example["system"]})
    
    messages.append({"role": "user", "content": example["instruction"] + 
                    ("\n\n" + example["input"] if example.get("input") else "")})
    messages.append({"role": "assistant", "content": example["output"]})
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return text

# Пример датасета на русском
raw_dataset = load_dataset("IlyaGusev/ru_turbo_alpaca", split="train")
# Или ваш датасет в формате jsonl с полями instruction/input/output

# Делим на train/test
dataset = raw_dataset.train_test_split(test_size=0.01, seed=42)  # 1% на тест достаточно
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# -------------------------- 4. ТРЕЙНЕР --------------------------
training_args = TrainingArguments(
    per_device_train_batch_size=BATCH_SIZE_PER_GPU,
    per_device_eval_batch_size=BATCH_SIZE_PER_GPU,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_ratio=WARMUP_RATIO,
    num_train_epochs=TOTAL_EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=5,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=42,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",           # или "wandb"
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    gradient_checkpointing=False,      # unsloth уже включил свою версию выше
    dataloader_num_workers=4,
    remove_unused_columns=False,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",                  # мы сами сформируем текст ниже
    max_seq_length=MAX_SEQ_LENGTH,
    args=training_args,
    formatting_func=formatting_func,            # ← вот здесь вся магия промптов
    packing=False,                              # для длинных контекстов лучше False
    dataset_kwargs={
        "add_special_tokens": False,            # важно для Qwen3
        "append_concat_token": False,
    },
)

print("Начинаем обучение Qwen3-14B с Unsloth...")
trainer.train()

# Сохраняем только LoRA-адаптер (всего ~300–500 МБ)
model.save_pretrained_merged(OUTPUT_DIR, tokenizer, save_method="lora")  # только LoRA
# Или полную модель (около 30 ГБ): save_method="merged_16bit"
print(f"Модель сохранена в {OUTPUT_DIR}")

# -------------------------- 5. ИНФЕРЕНС НА ТЕСТЕ ---------------------------
# Быстрая загрузка для инференса (в 4 раза быстрее обычного transformers!)
inference_model = FastLanguageModel.for_inference(model)  # включаем оптимизации

def generate_answer(instruction, input_text="", max_new_tokens=1024):
    messages = [
        {"role": "system", "content": "Ты полезный и честный помощник."},
        {"role": "user", "content": instruction + ("\n\n" + input_text if input_text else "")}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")
    
    outputs = inference_model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Обрезаем до ответа ассистента
    return response.split("assistant")[-1].strip()

# Пример генерации на тестовых примерах
test_examples = eval_dataset.select(range(50))
predictions = []

for ex in tqdm(test_examples, desc="Генерация предсказаний"):
    pred = generate_answer(ex["instruction"], ex.get("input", ""))
    predictions.append({
        "instruction": ex["instruction"],
        "input": ex.get("input", ""),
        "reference": ex["output"],
        "prediction": pred
    })

with open("qwen3_14b_predictions.json", "w", encoding="utf-8") as f:
    json.dump(predictions, f, ensure_ascii=False, indent=2)

print("Готово! Предсказания сохранены.")